In [1]:
def get_news(driver, category):

    data = {'title':[], 'agency':[], 'category':[], 'link':[]}

    current_page = 1
    
    while current_page < 6:
        try:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            news = soup.select('#timeline > ul > li')
            
            for item in news:
                title = item.select_one('strong > a').text
                agency = item.select_one('span.info_cp').text
                link = item.select_one('strong > a')['href']
                
                data['title'].append(title)
                data['agency'].append(agency)
                data['category'].append(category)
                data['link'].append(link)
            
            page_numbers = driver.find_elements(By.CSS_SELECTOR, '#timeline > div > div > a')
            driver.implicitly_wait(5)
            
            for p in page_numbers:
                if (p.text != '다음') and (int(p.text) == current_page + 1):
                    p.click()
                    
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#timeline > ul > li')))

                    current_page = current_page + 1
                    print(p.text, current_page)
                    break        

        except:
            pass
    
    df = pd.DataFrame(data)
    
    driver.close()

    return df

In [4]:
# Seleium 드라이버 생성
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 화면없이 실행

# 드라이버 서비스 생성
service = Service(ChromeDriverManager().install())

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=service, options=options)

url = "https://news.daum.net/"

driver.get(url)

category_tabs = driver.find_elements(By.CSS_SELECTOR, '#gnbContent > div > ul > li')

category_dict = {item.text:item for item in category_tabs}
category_dict['경제'].click()

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '#timeline > ul > li'))
)

# 데이터 수집
df_economy = get_news(driver, '경제')
print(len(df_economy))

df_economy.head()

2 2
3 3
4 4
5 5
6 6
140


,title,agency,category,link
0,5월 출생아수 2.7%↑ 두 달 연속 증가…혼인 건수는 21%↑,KBS,경제,https://v.daum.net/v/20240724124007227
1,"결혼 증가에…5월 태어난 아기, 8년반 만에 두달 연속 늘었다",중앙일보,경제,https://v.daum.net/v/20240724123551161
2,"길어지는 캐즘에…K-배터리, 줄줄이 투자 축소",연합뉴스TV,경제,https://v.daum.net/v/20240724123216130
3,'K-뷰티' 열풍에 中企 수출 3분기 연속 상승,전자신문,경제,https://v.daum.net/v/20240724122902096
4,지원책 효과 보나?…결혼·출산 늘었다,한국경제TV,경제,https://v.daum.net/v/20240724122730074


In [5]:
# 페이지 이동할 때 동적으로 기사 배열이 바뀌면서 일부 뉴스가 중복 수집되는 현상이 있습니다. 중복을 제거합니다. 
df_economy = df_economy.drop_duplicates()
len(df_economy)

60

In [6]:
# 데이터 저장
df_economy.to_csv('daum_economy_news.csv', index=False)